In [2]:
import pickle
import os.path as osp
import numpy as np
import torch
import pandas as pd

In [4]:
code_idx = pickle.load(open(osp.join('../new_data', 'code_indices', 'code_dict_pretrained.pkl'), 'rb'))

In [5]:
emb_name = list(code_idx.keys())[1:5089]

In [70]:
emb_name[-1]

'p_FIDA200'

In [36]:
diag_list = sorted([k.split('_')[1] for k, v in  code_idx.items() if 'd_' in k])
# proc_list = sorted([k for k, v in  code_idx.items() if 'pcs_' in k])
# drug_list = sorted([k for k, v in  code_idx.items() if 'p_' in k])

In [37]:
len(diag_list), len(proc_list), len(drug_list)

(863, 738, 3487)

In [38]:
len(diag_list) + len(proc_list) + len(drug_list)

5088

In [39]:
gpt_embs = pickle.load(open(osp.join('../new_data', 'gpt_emb', 'gpt4o_emb_all.pkl'), 'rb'))

In [42]:
gpt_embs.keys()

dict_keys(['003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '021', '023', '024', '027', '031', '032', '033', '034', '035', '036', '038', '039', '040', '041', '042', '045', '046', '047', '048', '049', '052', '053', '054', '055', '057', '058', '059', '061', '066', '070', '074', '075', '077', '078', '079', '082', '083', '084', '085', '086', '087', '088', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '104', '110', '111', '112', '114', '115', '116', '117', '118', '120', '121', '122', '123', '125', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '170', '171', '172', '173', '174', '175', '176', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '1

In [49]:
gpt_embs['003'].keys()

dict_keys(['answer', 'te3-small', 'te-ada002', 'te3-large'])

In [51]:
gpt_embs['003']['te3-large'].shape

(3072,)

In [40]:
# gpt_te3_large = pickle.load(open(osp.join('../new_data', 'gpt_emb', 'gpt4o_te3_small_v2.pkl'), 'rb'))

In [57]:
te3_small_emb = np.empty((0, 1536), dtype=np.float64)
for diag_code in diag_list:
    if diag_code in gpt_embs:
        te3_small_emb = np.vstack((te3_small_emb, gpt_embs[diag_code]['te3-small']))
    
te_ada002_emb = np.empty((0, 1536), dtype=np.float64)
for diag_code in diag_list:
    if diag_code in gpt_embs:
        te_ada002_emb = np.vstack((te_ada002_emb, gpt_embs[diag_code]['te-ada002']))
        
te3_large_emb = np.empty((0, 3072), dtype=np.float64)
for diag_code in diag_list:
    if diag_code in gpt_embs:
        te3_large_emb = np.vstack((te3_large_emb, gpt_embs[diag_code]['te3-large']))
        
te3_small_emb = torch.tensor(te3_small_emb)
te_ada002_emb = torch.tensor(te_ada002_emb)
te3_large_emb = torch.tensor(te3_large_emb)

In [48]:
te3_small_emb.shape

(863, 1536)

In [54]:
te3_large_emb.shape

(863, 3072)

In [55]:
te_ada002_emb.shape 

(863, 1536)

In [59]:
with open('../new_data/gpt_emb/gpt4o_te3_small_v3.pkl', 'wb') as f:
    pickle.dump(te3_small_emb, f)
    
with open('../new_data/gpt_emb/gpt4o_te3_large_v3.pkl', 'wb') as f:
    pickle.dump(te3_large_emb, f)

with open('../new_data/gpt_emb/gpt4o_te_ada002_v3.pkl', 'wb') as f:
    pickle.dump(te_ada002_emb, f)

In [71]:
te3_small_emb

tensor([[ 4.8839e-03, -2.5858e-02,  1.7648e-02,  ...,  3.4295e-02,
          4.7361e-03, -2.8317e-05],
        [-8.3356e-03, -3.3920e-03,  1.7945e-02,  ...,  2.0730e-02,
         -8.1763e-03,  1.4665e-02],
        [-3.6209e-02, -6.9713e-03, -2.6393e-02,  ...,  2.0331e-02,
          1.8711e-02,  1.8734e-02],
        ...,
        [ 2.9727e-03,  3.9190e-02,  7.7839e-02,  ..., -7.6428e-04,
         -1.4561e-02,  1.4193e-02],
        [-1.5293e-02,  2.1082e-02,  4.1426e-02,  ..., -1.0000e-02,
         -2.6034e-03,  1.4695e-02],
        [ 4.2850e-03,  4.0744e-02,  7.2135e-02,  ...,  1.5992e-03,
         -5.5184e-03, -5.6317e-03]], dtype=torch.float64)

In [7]:
test_emb = np.random.rand(5088, 256)

In [9]:
test_emb.shape

(5088, 256)

In [ ]:
col_name = ['dim_' + str(i) for i in range(256)]
align_col_name = ['code_name'] + col_name + ['code_type']
emb_type_list = [
            "diagnosis" if item.startswith("d_") else
            "procedure" if item.startswith("pcs") else
            "drug" if item.startswith("p_") else item
            for item in emb_name
        ]

    

In [11]:
a = zip(emb_name, test_emb, emb_type_list)

In [12]:
next(a)

('d_199',
 array([0.47601604, 0.54829628, 0.52161135, 0.59098613, 0.77821898,
        0.79769353, 0.29097114, 0.3015518 , 0.40832228, 0.56135703,
        0.34107822, 0.04708456, 0.7335524 , 0.42549833, 0.97451793,
        0.91328378, 0.72626026, 0.44160451, 0.68775473, 0.38346053,
        0.103529  , 0.19257913, 0.2126189 , 0.66248696, 0.63892196,
        0.40204617, 0.92451694, 0.50800701, 0.08271457, 0.21413212,
        0.11282315, 0.02814683, 0.80093702, 0.22345915, 0.81441798,
        0.80522557, 0.47776033, 0.46447949, 0.97507031, 0.25667596,
        0.61951632, 0.47862559, 0.85324582, 0.24276602, 0.82023593,
        0.29211953, 0.13758854, 0.47696183, 0.28322601, 0.25229735,
        0.90130809, 0.25312463, 0.97246488, 0.86541002, 0.07190353,
        0.67303528, 0.44496472, 0.32995375, 0.08343489, 0.53678458,
        0.00816072, 0.4932709 , 0.95976682, 0.75241512, 0.30013456,
        0.70196827, 0.45348601, 0.70388887, 0.67556338, 0.358963  ,
        0.59987959, 0.78672919, 0.9465

In [99]:
len(emb_type_list)

5088

In [95]:
emb_df = pd.DataFrame(test_emb, columns=col_name)
emb_df['code_name'] = emb_name
emb_df['code_type'] = emb_type_list
emb_df = emb_df[align_col_name]

In [96]:
emb_df

,code_name,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,...,dim_247,dim_248,dim_249,dim_250,dim_251,dim_252,dim_253,dim_254,dim_255,code_type
0,d_199,0.804380,0.337478,0.134356,0.301501,0.163948,0.149199,0.393272,0.729537,0.394584,...,0.002302,0.386698,0.866861,0.744989,0.218477,0.460273,0.752268,0.263852,0.987103,diagnosis
1,d_868,0.087939,0.580319,0.229595,0.741980,0.481744,0.125827,0.412947,0.869535,0.335687,...,0.865507,0.679908,0.819196,0.673240,0.866178,0.449362,0.650601,0.720910,0.748355,diagnosis
2,d_726,0.021507,0.159112,0.420108,0.389855,0.832266,0.723794,0.122833,0.632996,0.449529,...,0.988224,0.128563,0.357250,0.222908,0.452946,0.125976,0.093656,0.780825,0.133905,diagnosis
3,d_273,0.873144,0.688845,0.840101,0.874684,0.915637,0.762943,0.191961,0.205824,0.965415,...,0.175348,0.268895,0.099704,0.278727,0.907562,0.748109,0.880175,0.964395,0.460302,diagnosis
4,d_614,0.623680,0.093672,0.462862,0.884159,0.033772,0.369463,0.935049,0.780775,0.097590,...,0.596233,0.432377,0.656679,0.033967,0.583820,0.083549,0.923083,0.414722,0.733565,diagnosis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5083,p_GCSF480I,0.777731,0.270822,0.329952,0.755195,0.860009,0.155142,0.499303,0.719865,0.477727,...,0.475020,0.915245,0.742595,0.887284,0.875853,0.455894,0.196734,0.541858,0.591421,drug
5084,p_SIRO12L,0.296606,0.404655,0.677413,0.627204,0.618369,0.031856,0.192987,0.500208,0.320411,...,0.074066,0.561553,0.275567,0.378036,0.322811,0.298765,0.630074,0.032413,0.985496,drug
5085,p_TEST25P,0.925911,0.974513,0.917185,0.325972,0.419796,0.160445,0.659165,0.491524,0.807192,...,0.182638,0.906346,0.794811,0.174246,0.291465,0.218418,0.385519,0.777424,0.499751,drug
5086,p_CHOL750,0.769992,0.181303,0.420601,0.362401,0.616659,0.135045,0.289483,0.895322,0.199388,...,0.936741,0.773231,0.239245,0.120417,0.706186,0.112361,0.369227,0.614068,0.162385,drug


In [83]:
# 리스트 컴프리헨션으로 조건 처리
updated_data = [
    "diagnoses" if item.startswith("d_") and not item.startswith("diagnoses") else
    "procedures" if item.startswith("pcs") else
    "drug" if item.startswith("p_") else item
    for item in emb_name
]

In [87]:
np.unique(np.array(updated_data), return_counts=True)

(array(['diagnoses', 'drug', 'procedures'], dtype='<U10'),
 array([ 863, 3487,  738]))

In [89]:
updated_data[-1]

'drug'

In [13]:
from sklearn.datasets import load_iris

In [14]:
ds = load_iris(as_frame=True)

In [18]:
df = ds.data
df["target"] = ds.target
cols = df.columns.tolist()
